# Running the PGM-State Estimation with simulated measurements

In this notebook, we will run the PGM-State Estimation with simulated measurements.

## Create a dataset in Fuseki

Download and start an Apache Jena Fuseki server as described in <https://jena.apache.org/documentation/fuseki2/index.html>.
Create a new dataset in Fuseki with the name `test_dataset` and load the CGMES files into it's default graph.
By default the Fuseki-UI is available at <http://localhost:3030>.

For now, this repository does not contain CGMES datasets. Datasets may be added in the future.
Datasets can be generated using PowSyBl, or the Conformity Datasets from ENTSO-E can be used, respecting their license.

In [1]:
from cgmes2pgm_converter import CgmesDataset
from cgmes2pgm_converter.common import Profile

base_url = "http://localhost:3030/dataset_name"
dataset = CgmesDataset(
    base_url=base_url,
    # cim_namespace="http://iec.ch/TC57/2013/CIM-schema-cim16#",
    cim_namespace="http://iec.ch/TC57/CIM100#",
    graphs={
        Profile.OP: base_url + "/op",
        Profile.MEAS: base_url + "/meas",
        Profile.SV: base_url + "/sv",
    },
)

## Simulate measurements

To perform a state estimation, we need measurements, which may not be provided as in the ENTSO-E Conformity Datasets.
The `MeasurementBuilder` generates these measurements based on the provided SV Profile.

CGMES defines the OP Profile to specify the measurement infrastructure using the classes `Analog` and `AnalogValue`.
These objects are defined in the `/op` graph and are used to define the measurement infrastructure.
The concrete measurement values are defined in a separate `/meas` graph, since CGMES does not intend to store concrete measurement values in the OP Profile.
This allows the replacement of the `/meas` graph without the need to redefine the measurement infrastructure in the `/op` graph.

The values from the SV Profile are distorted using a normal distribution. Its standard deviation is defined in the provided `meas_ranges.yaml` file.
The standard deviation is chosen based on the voltage level of the bus and the type of measurement.


In [2]:
from cgmes2pgm_suite.measurement_simulation import MeasurementBuilder
from cgmes2pgm_suite.config import MeasurementSimulationConfigReader


reader = MeasurementSimulationConfigReader("meas_ranges.yaml")
config = reader.read()

builder = MeasurementBuilder(dataset, config)
builder.build_from_sv()

INFO     :: Building Voltage Measurements: 0.4194319248199463 seconds
INFO     :: Building Power Measurements: 2.3895022869110107 seconds


## Generate the PGM-Dataset

The `CgmesToPgmConverter` is used to convert the CGMES dataset into a PGM dataset.
Multiple parameters can be configured like limiting the model to a specific topological island using `cim:TopologicalIsland` from the SV-Profile.

The generated `extra_info` object defines additional information about the generated objects like their initial mRID, name, and type.


In [3]:
from cgmes2pgm_converter import CgmesToPgmConverter

converter = CgmesToPgmConverter(datasource=dataset)
input_data, extra_info = converter.convert()

The generated `input_data` and `extra_info` objects can be serialized to a JSON file using the package `power-grid-model-io`.

In [4]:
from power_grid_model_io.converters.pgm_json_converter import PgmJsonConverter

converter = PgmJsonConverter(destination_file="../out/model.json")
converter.save(input_data, extra_info)

## Run PGM state estimation

To run the PGM state estimation, the `StateEstimationWrapper` can be used, providing additional features such as calculating $J$ and $E(J)$, which can be used to evaluate the quality of the state estimation result.

In [5]:
from typing import cast
from cgmes2pgm_suite.state_estimation import (
    StateEstimationWrapper,
    StateEstimationResult,
)

state_estimation_wrapper = StateEstimationWrapper(
    input_data,
    extra_info,
)
result = cast(StateEstimationResult, state_estimation_wrapper.run())

INFO     :: State Estimation: 0.0014204978942871094 seconds


## Export the results

In addition to exporting results to JSON using `power-grid-model-io`, this package also supports exporting them in more human-readable formats, such as Excel.

The generated Excel file contains the following information:
- $J$ and $E(J)\pm 3\sigma$
- Number of bad data $|u_{measured}-u_{estimated}| > 3\sigma$
- Node Voltages and Power Flows
- List of all measurements
- Comparison of estimated values with an existing SV-Profile

In [6]:
from cgmes2pgm_suite.export import StesResultExcelExport

exporter = StesResultExcelExport(
    path="../out/output.xlsx",
    datasource=dataset,
    result=result,
    sv_comparison=False,
)
exporter.export()

True

## Create SV-Profile

The `SvProfileBuilder` is used to create a SV-Profile from the PGM dataset.
It writes the SV-Profile to the provided RDF-Graph. This includes objects of the following classes:
- `md:FullModel` based on the provided `model_info` object or created using default values
- `cim::TopologicalIsland` 
- `cim:SvVoltage` for all nodes
- `cim:SvPowerFlow` for all connected Terminals

In [7]:
from cgmes2pgm_suite.export import SvProfileBuilder, CgmesFullModel

model_info = CgmesFullModel(
    profile="http://entsoe.eu/CIM/StateVariables/4/1",
    description="dataset_name",
)

sv_builder = SvProfileBuilder(
    cgmes_dataset=dataset,
    pgm_dataset=result,
    target_graph=dataset.graphs[Profile.SV],
    model_info=model_info,
)
sv_builder.build(overwrite_existing=True)